In [1]:
library("xgboost")
library("Matrix")
library('Ckmeans.1d.dp')
library('lightgbm')



Attaching package: ‘lightgbm’


The following object is masked from ‘package:xgboost’:

    slice




## Read data and process data labels

In [2]:
time_matrix <- matrix(0,ncol = 3, nrow =4)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [3]:
data=read.csv('Monthly_features_matrix.csv')
data=data[,2:(dim(data)[2]-6)]

In [4]:
dim(data)

[1] 48000    21

In [5]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,12,1,12,0.9472661,1.912037e-08,0.872829,10.9548535,0.18758224,0.1214246,0.8404932,⋯,4,0.5701347,0.8835213,4.516663,0.06842530,0.42983270,-0.3962824,0.4604731,0.8293577,469
2,12,1,12,0.8085632,2.203382e-07,0.943755,4.7359892,0.09830677,0.1405957,0.4180888,⋯,8,0.7284204,0.7422968,3.837742,-0.32845599,0.18127459,-0.5668803,0.3576969,0.5860238,469
3,12,1,12,0.9992200,8.446236e-12,21.172689,4.0198303,0.36279469,0.4204365,0.5104123,⋯,7,0.1400863,0.9924302,9.220713,-0.09944540,0.10351393,-0.4913136,0.3353221,0.9154861,469
4,12,1,12,0.8687883,3.833819e-06,-7.295270,0.3739581,0.50800514,0.5935817,0.2357374,⋯,5,0.5748673,0.8699293,3.515017,-0.03200097,0.11979641,-0.5149001,0.3611336,0.3110420,82
5,12,1,12,0.7457338,1.006626e-05,4.869531,0.5621160,0.19177799,0.2581481,0.3207559,⋯,2,0.7126886,0.7436571,2.000761,-0.39225503,0.27488991,-0.6516841,0.5925659,0.1199568,112
6,12,1,12,0.9976576,3.100646e-10,13.985335,-12.7683661,0.50582153,0.2947603,0.4390163,⋯,4,0.2664074,0.9959503,9.486606,0.08404087,0.02687501,-0.4473514,0.2338179,0.9367465,666


In [6]:
dlist= load('Monthly_thetaf_datalist.RData')
datalist=eval(parse(text = dlist ))
res=datalist[[1]]
MASE=res[,,,6]
m=5

In [7]:
dim(MASE)

[1] 48000     5     4

In [8]:
whichmin<-function(x){
    minx=min(x[x>0])
    loc=which(x==minx)[1]-1
    loc
}

meanunique=function(x)
    {
    mean(unique(x))
}

In [9]:
nanum=c()
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    a=apply(count,1,min)
    if(sum(is.na(a))>0)
        {
        nanum=append(nanum,i)
    }
    }

In [10]:
nanum

[1]     4     5     7     8    41    42    43    56    58    59    61    62
   [13]    63    64    65    66    67    68    69    72    73    74    75    78
   [25]    79    81    82    83    84    85    86    87    88    91   103   104
   [37]   106   107   108   109   110   111   112   113   131   132   133   134
   [49]   135   136   137   141   143   147   153   154   155   160   161   162
   [61]   165   167   170   173   191   192   194   195   203   204   205   207
   [73]   215   216   217   218   219   220   221   222   223   226   227   228
   [85]   229   230   231   232   236   238   239   240   241   242   246   247
   [97]   248   252   253   254   255   256   257   258   259   261   262   263
  [109]   264   265   266   271   278   280   282   283   284   286   287   289
  [121]   290   292   293   295   296   297   298   300   303   313   314   315
  [133]   317   318   323   325   326   327   341   344   345   346   347   348
  [145]   349   354   365   366   373   384   388   389   391   396   398   399
  [157]   400   401   402   405   410   411   412   414   420   423   424   426
  [169]   427   434   435   436   439   440   441   447   460   461   462   463
  [181]   469   470   471   472   475   476   477   480   484   503   516   517
  [193]   544   564   566   574   575   576   577   586   591   594   610   612
  [205]   613   634   642   643   645   692   722   737   738   743   745   760
  [217]   765   773   774   777   782   790   800   801   808   843   845   875
  [229]   910   926   953   954   955   958  1064  1087  1112  1224  1232  1271
  [241]  1272  1273  1305  1352  1374  1379  1424  1425  1485  1512  1642  1665
  [253]  1684  1685  1723  1724  1729  1746  1768  1777  1781  1846  1849  1851
  [265]  1860  1861  1862  1913  1914  1949  1950  2040  2077  2130  2136  2139
  [277]  2140  2144  2147  2148  2149  2150  2151  2191  2192  2193  2196  2232
  [289]  2233  2267  2361  2373  2380  2383  2393  2404  2406  2416  2417  2418
  [301]  2419  2421  2462  2508  2509  2511  2582  2586  2587  2588  2589  2590
  [313]  2591  2592  2593  2594  2627  2628  2629  2643  2645  2648  2649  2650
  [325]  2674  2675  2678  2679  2680  2682  2685  2712  2721  2740  2741  2742
  [337]  2743  2744  2745  2746  2747  2748  2749  2750  2751  2752  2753  2754
  [349]  2755  2756  2757  2758  2759  2760  2761  2762  2763  2764  2765  2766
  [361]  2767  2768  2769  2770  2771  2772  2773  2774  2775  2776  2777  2778
  [373]  2779  2780  2781  2782  2783  2784  2785  2786  2787  2788  2789  2790
  [385]  2791  2792  2793  2794  2795  2796  2797  2798  2799  2800  2801  2802
  [397]  2803  2804  2805  2806  2836  2837  2838  2839  2840  2841  2842  2843
  [409]  2844  2845  2846  2847  2848  2849  2850  2851  2866  2867  2900  2901
  [421]  2911  2912  2913  2914  2915  2928  2952  2954  2955  2956  2964  2966
  [433]  2967  2968  2969  2986  2992  3006  3007  3011  3013  3014  3035  3036
  [445]  3037  3047  3050  3052  3053  3054  3069  3071  3072  3073  3074  3075
  [457]  3076  3078  3080  3081  3084  3116  3119  3139  3140  3141  3142  3148
  [469]  3161  3162  3163  3164  3165  3166  3198  3212  3213  3237  3267  3268
  [481]  3269  3270  3284  3329  3380  3385  3387  3390  3392  3466  3467  3468
  [493]  3469  3470  3542  3543  3544  3545  3546  3547  3548  3549  3550  3551
  [505]  3552  3553  3554  3555  3556  3557  3603  3614  3616  3619  3802  3891
  [517]  3903  3910  3911  3912  3913  3914  3915  3925  3936  3940  3948  4080
  [529]  4081  4082  4083  4084  4085  4086  4087  4088  4089  4090  4091  4092
  [541]  4093  4098  4100  4102  4103  4105  4108  4109  4110  4111  4112  4113
  [553]  4114  4115  4116  4117  4118  4120  4121  4122  4123  4124  4125  4126
  [565]  4127  4128  4129  4130  4131  4132  4133  4134  4136  4137  4138  4139
  [577]  4140  4141  4142  4143  4144  4145  4146  4147  4148  4149  4223  4224
  [589]  4225  4226  4230  4371  4427  4428  4429  4430  4431  4432  4433  4434
  [6

In [11]:
realbestmin=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    min_value=apply(count,1,min)
    if (max(min_value,na.rm = TRUE)==0){
        realbestmin[i,]=0
        }
    else
        {
        min_value[is.na(min_value)]=100 
        realbestmin[i,]= whichmin(min_value)
    }
    }

In [12]:
table(realbestmin)

realbestmin
    0     1     2     3     4 
10737  7155  6753 12733 10622 

In [13]:
realbestmean=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    mean_value=apply(count,1,meanunique)
    if (max(mean_value,na.rm = TRUE)==0){
        realbestmean[i,]=0
        }
    else
        {
        mean_value[is.na(mean_value)]=100 
        realbestmean[i,]= whichmin(mean_value)
    }
    }

In [14]:
realbestmean

3
4
3
3
0
1
0
0
0
4
2


In [15]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,12,1,12,0.9472661,1.912037e-08,0.872829,10.9548535,0.18758224,0.1214246,0.8404932,⋯,4,0.5701347,0.8835213,4.516663,0.06842530,0.42983270,-0.3962824,0.4604731,0.8293577,469
2,12,1,12,0.8085632,2.203382e-07,0.943755,4.7359892,0.09830677,0.1405957,0.4180888,⋯,8,0.7284204,0.7422968,3.837742,-0.32845599,0.18127459,-0.5668803,0.3576969,0.5860238,469
3,12,1,12,0.9992200,8.446236e-12,21.172689,4.0198303,0.36279469,0.4204365,0.5104123,⋯,7,0.1400863,0.9924302,9.220713,-0.09944540,0.10351393,-0.4913136,0.3353221,0.9154861,469
4,12,1,12,0.8687883,3.833819e-06,-7.295270,0.3739581,0.50800514,0.5935817,0.2357374,⋯,5,0.5748673,0.8699293,3.515017,-0.03200097,0.11979641,-0.5149001,0.3611336,0.3110420,82
5,12,1,12,0.7457338,1.006626e-05,4.869531,0.5621160,0.19177799,0.2581481,0.3207559,⋯,2,0.7126886,0.7436571,2.000761,-0.39225503,0.27488991,-0.6516841,0.5925659,0.1199568,112
6,12,1,12,0.9976576,3.100646e-10,13.985335,-12.7683661,0.50582153,0.2947603,0.4390163,⋯,4,0.2664074,0.9959503,9.486606,0.08404087,0.02687501,-0.4473514,0.2338179,0.9367465,666


In [16]:
set.seed(100)
index = sample(2,nrow(data),replace = TRUE,prob=c(0.7,0.3))

In [17]:
train_data=data[index==1,]
test_data=data[index==2,]
train_label_min=realbestmin[index==1,]
test_label_min=realbestmin[index==2,]
train_label_mean=realbestmean[index==1,]
test_label_mean=realbestmean[index==2,]

In [18]:
head(train_data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,12,1,12,0.9472661,1.912037e-08,0.872829,10.9548535,0.18758224,0.1214246,0.8404932,⋯,4,0.5701347,0.8835213,4.516663,0.06842530,0.42983270,-0.3962824,0.4604731,0.8293577,469
2,12,1,12,0.8085632,2.203382e-07,0.943755,4.7359892,0.09830677,0.1405957,0.4180888,⋯,8,0.7284204,0.7422968,3.837742,-0.32845599,0.18127459,-0.5668803,0.3576969,0.5860238,469
3,12,1,12,0.9992200,8.446236e-12,21.172689,4.0198303,0.36279469,0.4204365,0.5104123,⋯,7,0.1400863,0.9924302,9.220713,-0.09944540,0.10351393,-0.4913136,0.3353221,0.9154861,469
4,12,1,12,0.8687883,3.833819e-06,-7.295270,0.3739581,0.50800514,0.5935817,0.2357374,⋯,5,0.5748673,0.8699293,3.515017,-0.03200097,0.11979641,-0.5149001,0.3611336,0.3110420,82
5,12,1,12,0.7457338,1.006626e-05,4.869531,0.5621160,0.19177799,0.2581481,0.3207559,⋯,2,0.7126886,0.7436571,2.000761,-0.39225503,0.27488991,-0.6516841,0.5925659,0.1199568,112
6,12,1,12,0.9976576,3.100646e-10,13.985335,-12.7683661,0.50582153,0.2947603,0.4390163,⋯,4,0.2664074,0.9959503,9.486606,0.08404087,0.02687501,-0.4473514,0.2338179,0.9367465,666


In [19]:
end_time = Sys.time()

In [20]:
time_matrix[1,]=end_time-start_time

## Target the interval where the actual error is minimum

In [21]:
start_time = Sys.time()

In [22]:
dtrain_xg_min_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_min)) 
dtrain_xg_min_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)) )

In [23]:
dtrain_lg_min_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_min))
dtrain_lg_min_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)))

In [24]:
xgb_min_reg <- xgboost(data = dtrain_xg_min_reg, nround=100)

[1]	train-rmse:1.774529 
[2]	train-rmse:1.527482 
[3]	train-rmse:1.385812 
[4]	train-rmse:1.304063 
[5]	train-rmse:1.256667 
[6]	train-rmse:1.230013 
[7]	train-rmse:1.212804 
[8]	train-rmse:1.201410 
[9]	train-rmse:1.194380 
[10]	train-rmse:1.187842 
[11]	train-rmse:1.181965 
[12]	train-rmse:1.177312 
[13]	train-rmse:1.174613 
[14]	train-rmse:1.171022 
[15]	train-rmse:1.169699 
[16]	train-rmse:1.167278 
[17]	train-rmse:1.161549 
[18]	train-rmse:1.159090 
[19]	train-rmse:1.158110 
[20]	train-rmse:1.154045 
[21]	train-rmse:1.153274 
[22]	train-rmse:1.151192 
[23]	train-rmse:1.148118 
[24]	train-rmse:1.143621 
[25]	train-rmse:1.139199 
[26]	train-rmse:1.136517 
[27]	train-rmse:1.132021 
[28]	train-rmse:1.130515 
[29]	train-rmse:1.129499 
[30]	train-rmse:1.127008 
[31]	train-rmse:1.124720 
[32]	train-rmse:1.120314 
[33]	train-rmse:1.119149 
[34]	train-rmse:1.118219 
[35]	train-rmse:1.117364 
[36]	train-rmse:1.112983 
[37]	train-rmse:1.111099 
[38]	train-rmse:1.109533 
[39]	train-rmse:1.107

In [25]:
xgb_min_cl <- xgboost(data = dtrain_xg_min_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.490503 
[2]	train-mlogloss:1.415740 
[3]	train-mlogloss:1.360815 
[4]	train-mlogloss:1.318864 
[5]	train-mlogloss:1.285430 
[6]	train-mlogloss:1.258561 
[7]	train-mlogloss:1.236793 
[8]	train-mlogloss:1.218158 
[9]	train-mlogloss:1.201335 
[10]	train-mlogloss:1.187535 
[11]	train-mlogloss:1.175535 
[12]	train-mlogloss:1.164508 
[13]	train-mlogloss:1.154716 
[14]	train-mlogloss:1.147004 
[15]	train-mlogloss:1.138415 
[16]	train-mlogloss:1.129776 
[17]	train-mlogloss:1.121399 
[18]	train-mlogloss:1.114700 
[19]	train-mlogloss:1.109256 
[20]	train-mlogloss:1.104372 
[21]	train-mlogloss:1.099045 
[22]	train-mlogloss:1.092880 
[23]	train-mlogloss:1.087443 
[24]	train-mlogloss:1.082982 
[25]	train-mlogloss:1.075797 
[26]	train-mlogloss:1.069244 
[27]	train-mlogloss:1.064059 
[28]	train-mlogloss:1.059690 
[29]	train-mlogloss:1.057139 
[30]	train-mlogloss:1.051858 
[31]	train-mlogloss:1.047186 
[32]	train-mlogloss:1.041892 
[33]	train-mlogloss:1.038021 
[34]	train-mlogloss

In [26]:
lgb_min_reg <- lgb.train(data = dtrain_lg_min_reg, nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010143 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4103
[LightGBM] [Info] Number of data points in the train set: 33791, number of used features: 18
[LightGBM] [Info] Start training from score 2.113551


In [27]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_min_cl <- lgb.train(data = dtrain_lg_min_cl,nrounds = 100,params=params)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015674 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4103
[LightGBM] [Info] Number of data points in the train set: 33791, number of used features: 18
[LightGBM] [Info] Start training from score -1.495869
[LightGBM] [Info] Start training from score -1.912959
[LightGBM] [Info] Start training from score -1.957639
[LightGBM] [Info] Start training from score -1.328318
[LightGBM] [Info] Start training from score -1.504225


In [28]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

## Target the interval where the average error is minimum

In [29]:
start_time = Sys.time()

In [30]:
dtrain_xg_mean_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_mean)) 
dtrain_xg_mean_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)) )

In [31]:
dtrain_lg_mean_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_mean))
dtrain_lg_mean_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)))

In [32]:
xgb_mean_reg <- xgboost(data = dtrain_xg_mean_reg, nround=100)

[1]	train-rmse:1.583420 
[2]	train-rmse:1.404890 
[3]	train-rmse:1.301100 
[4]	train-rmse:1.242215 
[5]	train-rmse:1.206081 
[6]	train-rmse:1.185006 
[7]	train-rmse:1.171001 
[8]	train-rmse:1.160842 
[9]	train-rmse:1.151170 
[10]	train-rmse:1.145927 
[11]	train-rmse:1.139525 
[12]	train-rmse:1.134425 
[13]	train-rmse:1.131126 
[14]	train-rmse:1.129025 
[15]	train-rmse:1.122710 
[16]	train-rmse:1.117563 
[17]	train-rmse:1.115900 
[18]	train-rmse:1.113229 
[19]	train-rmse:1.110484 
[20]	train-rmse:1.108259 
[21]	train-rmse:1.107069 
[22]	train-rmse:1.102347 
[23]	train-rmse:1.097971 
[24]	train-rmse:1.095675 
[25]	train-rmse:1.091429 
[26]	train-rmse:1.087912 
[27]	train-rmse:1.084464 
[28]	train-rmse:1.083383 
[29]	train-rmse:1.081554 
[30]	train-rmse:1.077401 
[31]	train-rmse:1.074492 
[32]	train-rmse:1.071101 
[33]	train-rmse:1.066755 
[34]	train-rmse:1.065117 
[35]	train-rmse:1.060366 
[36]	train-rmse:1.058544 
[37]	train-rmse:1.058013 
[38]	train-rmse:1.054742 
[39]	train-rmse:1.051

In [33]:
xgb_mean_cl <- xgboost(data = dtrain_xg_mean_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.489088 
[2]	train-mlogloss:1.414437 
[3]	train-mlogloss:1.361150 
[4]	train-mlogloss:1.320539 
[5]	train-mlogloss:1.288091 
[6]	train-mlogloss:1.261376 
[7]	train-mlogloss:1.239953 
[8]	train-mlogloss:1.221174 
[9]	train-mlogloss:1.204880 
[10]	train-mlogloss:1.190283 
[11]	train-mlogloss:1.177266 
[12]	train-mlogloss:1.164744 
[13]	train-mlogloss:1.154507 
[14]	train-mlogloss:1.147203 
[15]	train-mlogloss:1.138113 
[16]	train-mlogloss:1.129915 
[17]	train-mlogloss:1.122113 
[18]	train-mlogloss:1.113966 
[19]	train-mlogloss:1.106572 
[20]	train-mlogloss:1.101465 
[21]	train-mlogloss:1.096847 
[22]	train-mlogloss:1.090182 
[23]	train-mlogloss:1.084265 
[24]	train-mlogloss:1.079175 
[25]	train-mlogloss:1.074928 
[26]	train-mlogloss:1.071325 
[27]	train-mlogloss:1.067829 
[28]	train-mlogloss:1.063765 
[29]	train-mlogloss:1.060621 
[30]	train-mlogloss:1.055569 
[31]	train-mlogloss:1.051010 
[32]	train-mlogloss:1.047895 
[33]	train-mlogloss:1.043676 
[34]	train-mlogloss

In [34]:
lgb_mean_reg <- lgb.train(data = dtrain_lg_mean_reg,nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014093 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4103
[LightGBM] [Info] Number of data points in the train set: 33791, number of used features: 18
[LightGBM] [Info] Start training from score 1.734308


In [35]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_mean_cl <- lgb.train(data = dtrain_lg_mean_cl,params=params,nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005366 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4103
[LightGBM] [Info] Number of data points in the train set: 33791, number of used features: 18
[LightGBM] [Info] Start training from score -1.194968
[LightGBM] [Info] Start training from score -1.857405
[LightGBM] [Info] Start training from score -1.758894
[LightGBM] [Info] Start training from score -1.418258
[LightGBM] [Info] Start training from score -2.064841


In [36]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

## predict

In [37]:
start_time = Sys.time()

In [38]:
alldatalgb <- lgb.Dataset(data = as.matrix(data))
alldataxgb <- xgb.DMatrix(data = as.matrix(data))


In [39]:
xgbregmin=predict(xgb_min_reg,alldataxgb)
xgbclsmin=predict(xgb_min_cl,alldataxgb)
lgbregmin=predict(lgb_min_reg,as.matrix(data))
lgbclsmin=predict(lgb_min_cl,as.matrix(data))

In [40]:
xgbregmin

[1]  3.1990168095  2.6809048653  3.4858512878  1.7220181227  1.2126711607
    [6]  1.2907420397  1.9551184177  2.1304891109  0.3222190738  2.7682640553
   [11]  2.2812495232  4.0921273232  3.0565943718  3.2380349636  2.4612312317
   [16]  0.6144708991  2.9916331768  1.0618001223  2.0784749985  2.4064974785
   [21]  1.9150679111  2.3773090839  2.1406445503  3.8902375698  4.0268154144
   [26]  3.8802287579  3.2623300552  2.8757493496  3.1704123020  3.4971005917
   [31]  2.7811133862  3.4496350288  3.4594957829  3.3407223225  3.4575462341
   [36]  3.4573402405  2.7907490730  3.2969627380  3.1720354557  3.0899744034
   [41]  0.5008423328  1.2551451921  1.6691603661  2.5989243984  2.8647205830
   [46]  1.0389239788  0.8417191505  2.0107080936  0.7261632085  0.9894865751
   [51]  0.6310175657  3.5658824444  3.8001277447  0.6816794872  3.6006608009
   [56]  1.3139455318  2.8288786411  0.6900476217  1.1537709236  2.3111252785
   [61]  2.0479683876  1.2504256964  2.3019087315  0.6361717582  0.3865589797
   [66]  1.0770533085  0.6407647729  0.9158806801  1.7004052401  0.3267985284
   [71]  0.9573054910  0.6131169796  1.7204802036  1.7274641991  1.1700661182
   [76]  1.7211004496  2.9921114445  1.1410979033  1.2967578173  0.5531336069
   [81]  0.7896506190  1.0720775127  0.9660886526  2.0080313683  1.6199792624
   [86]  1.3964477777  1.7536643744  1.2944699526  2.9596519470  2.2726509571
   [91]  2.1367812157  3.5485577583  4.2429127693  3.0403108597  2.3424198627
   [96]  2.1364972591  2.6014015675  3.8837432861  2.8074576855  2.4580397606
  [101]  3.2967822552  2.6102735996  1.7996017933  1.6708182096  3.8231780529
  [106]  2.4872713089  2.7982888222  2.4860653877  1.3634204865  3.1635355949
  [111]  1.1958495378  2.7730731964  2.5643310547  4.0956482887  2.2225158215
  [116]  3.4258105755  3.8828649521  4.0562305450  3.8994455338  4.1225056648
  [121]  3.8991873264  4.0025653839  1.6274784803  3.7190337181  3.0048279762
  [126]  2.8159317970  3.1572811604  3.1322433949  3.2107167244  3.2702348232
  [131]  2.1859004498  0.8737293482  2.2219867706  2.5447096825  1.2032315731
  [136]  1.6965452433  1.5985932350  3.4920263290  3.5059685707  3.8329048157
  [141]  1.4543001652  3.5461518764  1.4767521620  3.8500628471  4.2927985191
  [146]  2.1866831779  1.5187009573  3.8961997032  4.0301556587  3.8902084827
  [151]  3.8424656391  3.4012594223  0.9550024867  0.9536427259  2.1890459061
  [156]  3.7786521912  3.3917660713  4.1190271378  3.3493344784  1.8890367746
  [161]  2.0615200996  2.1158835888  3.0982644558  2.8621656895  0.5359702706
  [166]  4.0437912941  0.8381206989  3.8001985550  3.8455731869  1.6186176538
  [171]  2.8429954052  3.4052438736  2.0316739082  3.1419067383  3.4893388748
  [176]  3.4301836491  3.2625684738  3.1827409267  2.4762721062  4.1870913506
  [181]  3.5242128372  3.1939911842  3.9017708302  3.5135707855  2.3566765785
  [186]  4.1969466209  3.6844482422  3.9760468006  2.9686150551  3.5955302715
  [191]  1.6045739651  1.1458042860  3.6698367596  1.5043115616  1.0558611155
  [196]  3.0038177967  3.4345450401  3.6058862209  3.4829373360  3.9211404324
  [201]  2.4266550541  3.6272113323  3.0580658913  1.8391190767  1.1527036428
  [206]  3.9142076969  1.7293806076  3.4545090199  2.8920927048  3.8498315811
  [211]  3.8384351730  3.5302429199  2.2686648369  3.5855813026  1.1032179594
  [216]  0.6270703077  1.3040609360  0.8274918795  1.5454511642  0.8291490078
  [221]  1.7301086187  1.5852837563  0.9223300815  2.9161753654  2.2531440258
  [226]  0.8451516628  0.9374137521  1.1829185486  0.7111737132  1.8847696781
  [231]  1.1829185486  2.8145205975  3.2692050934  3.7063136101  2.7916388512
  [236]  1.8230968714  3.0870678425  1.9068468809  1.4235016108  1.6848890781
  [241]  1.5560872555  1.9738726616  2.9460723400  1.6706517935  3.0329449177
  [246]  2.5879039764  1.7276108265  1.3154993057  2.0041429996  2.6985797882
  [251]  3.4688112736  1.3779069185  2.2372670174  1.9135572910  1.2607159615
  [256]  1.2618376017  1.7

In [41]:
xgbclsmin

[1] 4 4 4 3 0 1 3 3 0 4 2 4 4 4 1 0 3 0 3 3 2 2 0 4 4 4 4 3 3 3 3 4 3 4 4 3
   [37] 4 3 3 3 0 1 1 4 0 0 0 0 0 1 0 4 4 0 4 3 4 0 0 3 1 0 3 0 0 0 0 0 3 0 0 0
   [73] 3 2 0 0 4 0 0 0 0 0 3 3 3 0 3 3 4 4 3 3 4 4 3 4 4 4 4 0 4 4 1 2 4 3 3 3
  [109] 0 3 0 3 3 4 4 4 4 4 4 4 4 4 1 4 3 4 4 4 4 4 3 3 3 3 0 1 2 4 4 4 1 4 1 4
  [145] 4 4 3 4 4 4 4 4 0 0 3 4 4 4 4 3 3 0 4 4 0 4 1 4 4 0 3 4 3 3 3 4 4 3 3 4
  [181] 4 4 4 4 4 4 4 4 4 4 1 0 3 1 0 3 3 4 4 4 3 4 3 3 0 4 3 4 3 4 4 4 3 4 0 0
  [217] 0 0 0 0 0 3 0 4 2 3 0 0 0 3 0 3 3 3 4 2 3 3 1 3 0 1 4 0 4 3 1 3 4 4 4 3
  [253] 2 2 0 3 3 3 0 3 2 2 3 0 3 3 3 4 4 4 0 4 4 4 3 4 4 3 3 0 4 3 1 3 4 2 0 4
  [289] 0 2 4 1 3 3 1 3 1 0 4 3 1 0 0 4 4 4 3 3 3 4 3 0 0 0 0 3 1 3 2 4 3 4 0 0
  [325] 0 3 0 4 4 4 4 2 4 4 4 4 4 2 3 4 0 4 4 4 0 3 3 1 0 4 4 4 4 0 3 3 1 1 4 4
  [361] 4 4 0 4 0 0 4 4 4 4 2 4 3 4 4 0 2 3 4 1 4 3 2 0 4 2 4 3 3 0 3 4 4 2 2 3
  [397] 4 1 0 3 3 2 4 4 2 4 2 3 4 0 0 3 2 3 4 4 2 4 4 3 4 4 4 0 1 3 3 4 3 3 3 4
  [433] 4 0 3 2 3 4 0 2 1 0 4 4 4 4 3 1 0 4 4 4 4 4 4 3 4 4 0 3 3 3 2 0 3 4 3 4
  [469] 3 4 3 3 4 3 3 0 1 2 0 2 4 4 4 3 4 3 3 0 4 3 4 1 3 4 4 4 4 0 4 0 1 3 2 2
  [505] 2 2 2 4 4 4 4 2 4 1 4 0 3 3 4 4 4 4 4 4 0 4 0 0 4 4 4 3 2 3 2 4 4 2 2 1
  [541] 2 2 4 0 4 4 4 2 2 4 4 0 4 3 4 0 4 2 1 2 2 2 4 3 4 2 4 2 0 3 4 4 4 3 0 0
  [577] 0 0 1 3 2 3 3 3 4 3 2 2 0 3 3 4 4 2 4 4 4 4 4 4 4 4 3 2 2 4 4 4 4 3 0 3
  [613] 3 4 3 4 2 4 1 2 3 4 0 4 3 4 3 4 4 2 4 3 3 3 4 4 4 4 3 3 4 4 3 0 3 4 4 4
  [649] 0 4 0 1 4 4 4 3 0 4 4 0 4 3 4 4 2 4 4 4 4 0 1 3 3 4 1 4 4 1 4 0 4 1 1 4
  [685] 0 3 0 4 3 0 0 0 0 4 3 3 4 3 4 4 0 4 4 4 4 0 0 0 4 0 0 3 3 4 0 0 4 4 4 4
  [721] 4 3 4 2 0 0 4 1 4 4 1 3 3 4 4 1 3 0 3 4 3 1 0 4 3 0 4 3 4 0 0 4 3 0 0 0
  [757] 4 1 4 3 3 4 4 4 1 4 4 4 4 0 4 4 3 3 4 3 3 4 1 1 1 0 3 0 4 4 4 4 4 1 3 4
  [793] 1 3 4 3 0 0 0 2 2 4 4 0 4 1 4 0 0 4 4 4 3 4 3 4 0 4 0 0 0 4 0 0 4 4 4 4
  [829] 0 4 0 4 4 3 4 1 4 3 4 4 4 4 0 4 3 4 4 4 3 4 4 4 1 4 3 3 4 4 3 3 3 4 3 3
  [865] 4 3 4 4 3 4 0 3 3 4 3 4 4 3 3 4 4 4 4 3 2 4 1 4 0 4 4 3 3 2 4 3 0 3 4 0
  [901] 4 1 4 3 4 4 1 4 4 0 3 4 4 0 3 4 4 0 4 4 4 4 4 4 4 3 0 4 4 0 0 4 1 4 0 3
  [937] 2 4 4 4 4 1 0 4 4 0 3 1 0 4 2 4 2 2 2 1 4 3 0 1 4 0 4 3 3 0 4 4 4 0 0 1
  [973] 0 4 4 4 1 1 0 3 4 4 4 4 4 4 4 4 4 4 1 3 4 1 1 0 2 2 4 0 4 4 0 4 4 4 0 4
 [1009] 4 4 0 1 3 2 4 4 1 3 4 4 2 4 4 0 4 4 3 4 4 1 0 1 0 1 4 4 4 1 3 4 2 0 4 4
 [1045] 4 4 4 1 0 2 1 4 4 2 4 0 4 0 0 4 0 0 4 0 4 4 4 4 4 0 1 4 4 3 4 4 0 2 0 4
 [1081] 0 4 4 4 4 0 1 4 4 4 4 4 4 2 2 1 1 3 0 2 1 1 1 3 4 4 3 4 4 4 4 0 3 4 3 0
 [1117] 3 4 0 1 4 0 4 4 3 4 4 4 0 0 4 0 3 4 0 4 4 0 3 4 4 3 2 3 4 3 0 3 3 4 3 1
 [1153] 3 0 4 3 4 4 4 3 0 0 0 4 0 3 0 4 0 4 4 4 3 3 4 4 4 3 2 0 0 3 4 3 0 4 3 4
 [1189] 4 4 3 3 4 0 3 3 3 0 3 0 1 0 4 4 1 0 0 3 0 0 0 4 0 1 4 0 4 4 0 4 4 4 4 3
 [1225] 3 4 4 3 1 0 4 0 4 3 4 4 4 0 4 1 4 0 3 3 4 4 4 3 4 4 3 0 4 1 4 4 4 3 0 0
 [1261] 4 4 4 0 2 3 4 4 4 4 0 3 3 2 4 0 0 0 4 4 4 0 0 0 3 4 4 0 1 0 0 4 4 4 4 3
 [1297] 4 4 0 4 4 4 3 0 2 4 4 3 3 3 4 4 4 4 3 4 4 4 4 4 0 4 1 3 3 1 4 3 0 4 4 3
 [1333] 4 4 4 4 4 0 0 4 4 0 0 4 4 2 4 3 4 0 4 0 3 4 3 4 4 3 4 0 0 4 0 4 4 4 4 4
 [1369] 4 4 4 3 4 0 0 4 0 4 0 4 4 4 4 4 4 4 0 0 0 4 0 0 4 4 4 4 4 3 0 3 3 0 0 3
 [1405] 3 4 4 4 4 4 4 2 0 3 4 4 3 2 0 4 1 3 0 0 1 4 3 2 3 3 0 4 1 0 4 4 4 4 0 4
 [1441] 4 4 3 0 4 0 0 1 3 4 1 1 3 3 4 4 4 1 3 3 1 1 3 0 4 4 0 4 2 4 0 0 4 4 4 4
 [1477] 3 0 4 3 4 0 3 4 2 0 4 0 3 3 3 0 0 4 4 3 3 4 3 3 4 4 0 4 4 3 0 4 4 3 3 0
 [1513] 4 4 3 4 4 4 4 4 3 0 0 1 4 4 3 3 0 0 4 2 4 3 3 3 3 1 4 3 4 1 0 3 4 4 3 0
 [1549] 4 2 4 4 0 1 1 0 3 4 4 4 4 0 4 0 0 4 3 1 4 0 4 4 3 0 3 0 4 4 4 4 0 4 4 1
 [1585] 0 0 0 3 0 4 0 4 0 0 4 4 3 4 4 3 4 4 4 4 4 4 4 0 4 4 3 0 0 0 4 0 0 0 0 4
 [1621] 4 4 4 4 4 0 3 4 0 0 3 4 4 0 4 1 4 3 1 0 2 3 0 0 4 0 0 3 4 0 4 4 4 1 0 1
 [1657] 4 4 4 3 4 4 4 4 1 3 4 4 4 3 4 4 0 0 4 4 4 4 4 3 3 4 1 0 0 1 4 2 4 2 4 0
 [1693] 1 4 0 3 3 0 0 3 3 3 4 4 4 4 4 0 4 0 3 0 0 4 4 3 0 0 0 4 4 4 1 3 4 4 4 4
 [1729] 3 3 3 4 4 0 0 4 4 4 0 2 2 0 4 4 4 2 4 4 4 3 4 4 3 3 3 3 4 4 4 3 4 4 0 4
 [1765] 4 4 1 3 4 4 4 4 3 0 3 3 1 4 0 0 1 4 4 3 1 4 4 4 0 4 4 4 1 0 4 3 3 0 0 4
 [18

In [42]:
lgbregmin

[1] 2.6253762 2.5999735 3.5233965 1.7609768 1.8720644 2.1045076 1.9208681
    [8] 2.1856720 1.4871441 2.4467500 2.5145159 2.2888717 2.5204012 2.1104793
   [15] 2.5426101 2.1189691 2.6335231 1.6710911 2.1144791 2.5083758 2.0192253
   [22] 2.0515469 2.0984354 3.5358421 3.8927075 3.5833118 3.1662673 3.2280638
   [29] 3.2347404 3.3586337 2.7869176 3.3946671 3.3660960 3.2791917 3.1309153
   [36] 3.2973145 2.5713628 3.3831273 3.1664080 2.9521604 1.7135835 1.3940675
   [43] 1.4565902 2.0331775 1.3749855 1.2854087 1.0995525 1.6190990 1.0786078
   [50] 1.6910500 1.2692589 3.5557805 2.8393893 1.8569336 3.5864853 1.9365246
   [57] 2.3071058 1.0127565 1.1711068 1.9187404 1.8732645 1.7584272 2.3837228
   [64] 0.9937873 0.9953223 1.5930540 0.8451595 1.2553720 1.9501992 1.3733340
   [71] 1.4935964 0.8468253 1.7276362 1.4914488 1.5372155 2.3321071 2.4026323
   [78] 0.9846588 1.3932671 1.3471320 0.5574442 1.4675717 1.3729596 1.9719747
   [85] 1.8567254 1.7615932 1.9882986 1.9237446 2.5755339 2.3200060 1.9533097
   [92] 3.5572322 3.9629792 2.9094326 2.6963743 2.4839597 2.4616010 3.4985845
   [99] 2.6248106 1.7118190 3.1985775 2.6248252 1.6488896 1.6446469 3.5166748
  [106] 1.8178405 2.3192640 1.8593538 1.4310920 2.5899473 1.4201699 2.5695179
  [113] 2.3594945 3.8130267 2.2714727 3.4345727 3.7963952 3.8463558 3.8444979
  [120] 3.8774145 3.8703812 3.6820593 3.5049822 3.2787705 2.7444692 2.8865658
  [127] 3.2420349 2.9727486 3.2203898 3.1712666 1.6941158 1.3889692 1.5175034
  [134] 1.7123593 1.2650551 1.6502708 1.4817608 3.4006898 3.3599427 3.3669506
  [141] 1.5491277 3.4798689 1.5162486 3.5334753 3.7618987 2.7774280 1.5761003
  [148] 3.8461207 3.8149498 3.8720889 3.7943896 3.5791718 0.9624638 0.9580424
  [155] 1.9449391 3.4985368 3.5000873 3.9229066 3.4275819 1.8101811 1.8419516
  [162] 1.4913252 2.8572691 2.8909568 1.2970147 3.7337319 0.9097103 3.3663680
  [169] 3.7039685 1.4191676 3.3517294 3.6730221 1.9246814 3.2812372 3.3255682
  [176] 3.4642297 3.2104383 3.3312033 2.7467870 3.9264111 3.5274119 3.0948134
  [183] 3.4360079 3.3514829 2.1114798 3.9954873 3.5038929 3.9021044 3.4228093
  [190] 3.4965767 1.7175231 1.1787316 3.8782601 1.3223282 1.2298101 3.7545581
  [197] 3.5549991 3.3202392 2.9978898 3.6677869 2.3507090 3.4936276 2.4828337
  [204] 1.7058998 1.4373070 3.6540971 1.3776035 3.5993260 3.2194155 3.8173937
  [211] 4.0486350 3.3719871 2.6108286 3.3004601 1.3810282 1.2652180 1.1167756
  [218] 1.0997312 1.5829047 1.1178363 0.9976065 1.2313537 1.4478436 2.6584746
  [225] 2.7841615 1.4188644 1.4478436 1.4144192 1.2076052 2.0855246 1.4144192
  [232] 2.3778714 3.4526291 3.5840188 2.7328075 1.9952688 3.3835380 1.9538976
  [239] 1.9081606 1.8196897 1.5030266 2.0450377 2.9651157 1.8857587 3.0680633
  [246] 2.0166812 1.4686561 1.4176650 2.2390453 2.9386052 3.4551542 1.4178283
  [253] 2.1760905 1.9690896 1.0261749 1.3496142 1.4990571 1.5013529 1.3818342
  [260] 2.7918170 1.1733544 1.7117126 1.6718581 1.2839698 2.1885652 2.2735477
  [267] 3.1305354 3.3860075 3.3238247 3.2081919 1.5447262 2.3012847 2.8960006
  [274] 2.9505589 2.6366245 3.3038364 2.0640465 1.6272969 2.5790105 1.3373734
  [281] 2.0733728 1.6466937 1.0408445 1.4935978 2.7126489 1.5832413 1.3631697
  [288] 3.1604256 1.4950768 1.7925277 2.4789521 1.8820305 2.1046044 1.9954793
  [295] 1.0274586 1.2068214 0.9960740 1.2814192 2.0822284 2.0547165 1.7386966
  [302] 1.7769025 1.5771647 3.5065738 3.2690150 2.9150458 3.9440544 3.5858518
  [309] 3.6193758 2.5806429 2.2319037 2.0918062 1.5936709 1.1026116 1.4581637
  [316] 3.3480318 1.4885373 2.0142966 3.2625714 2.8380509 3.3548379 2.7506490
  [323] 1.0336306 1.6813412 1.6094609 1.9665042 1.3019252 3.7582723 3.7842525
  [330] 2.4136833 1.9394542 3.0074553 3.2720961 3.3399762 3.0232056 3.0744708
  [337] 2.3868145 2.8780257 3.5180457 3.4006785 1.3119742 3.8863259 2.9544401
  [344] 2.6610425 1.4559358 1.7722718 2.8082921 1.5371687 1.4489158 3.0179159
  [351] 3.5254499 3.4449887 3.4822441 1.4707849 2.9411475 3.5482533 2.3842255
  [358] 2.4713001 2.839049

In [43]:
lgbclsmin

0.153984362,0.044648002,0.035243198,0.15892745,6.071970e-01
0.194116678,0.039930166,0.038106678,0.09815778,6.296887e-01
0.037309994,0.010964159,0.038786140,0.27702951,6.359102e-01
0.237718668,0.105382161,0.107538045,0.54927787,8.325327e-05
0.273046074,0.117571042,0.221650881,0.38760883,1.231690e-04
0.124107895,0.343356292,0.036633605,0.17818781,3.177144e-01
0.147800527,0.154091892,0.218758250,0.47923731,1.120189e-04
0.087186169,0.179658135,0.167226372,0.56579900,1.303199e-04
0.578884012,0.119748524,0.059493967,0.05864262,1.832309e-01
0.333862570,0.051272025,0.027262615,0.16349668,4.241061e-01
0.121456651,0.113153729,0.311178209,0.14292793,3.112835e-01


In [44]:
datalength=dim(data)[1]
lgbclsm=matrix(lgbclsmin,m,datalength)
lgbclsminr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsminr[i,]=which.max(lgbclsm[,i])
}
lgbclsminr=lgbclsminr-1

In [45]:
head(lgbclsminr)

4
3
3
2
2
1


In [46]:
xgbregmean=predict(xgb_mean_reg,alldataxgb)
xgbclsmean=predict(xgb_mean_cl,alldataxgb)
lgbregmean=predict(lgb_mean_reg,as.matrix(data))
lgbclsmean=predict(lgb_mean_cl,as.matrix(data))

In [47]:
lgbclsm=matrix(lgbclsmean,m,datalength)
lgbclsmeanr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsmeanr[i,]=which.max(lgbclsm[,i])
}
lgbclsmeanr=lgbclsmeanr-1

In [48]:
preallmin=cbind(xgbclsmin,xgbregmin)
preallmin=cbind(preallmin,lgbclsminr)
preallmin=cbind(preallmin,lgbregmin)

In [49]:
colnames(preallmin)=c('xgbclsmin','xgbregmin','lgbclsmin','lgbregmin')
head(preallmin)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
4,3.199017,4,2.625376
4,2.680905,3,2.599974
4,3.485851,3,3.523396
3,1.722018,2,1.760977
0,1.212671,2,1.872064
1,1.290742,1,2.104508


In [50]:
preallmean=cbind(xgbclsmean,xgbregmean)
preallmean=cbind(preallmean,lgbclsmeanr)
preallmean=cbind(preallmean,lgbregmean)

In [51]:
head(preallmean)

xgbclsmean,xgbregmean,,lgbregmean
3,2.500392,3,2.327374
4,1.949847,3,2.315179
4,3.251905,1,3.333878
3,1.482522,0,1.434231
0,1.476272,1,1.558648
1,1.332238,2,1.853177


In [52]:
colnames(preallmean)=c('xgbclsmean','xgbregmean','lgbclsmean','lgbregmean')
head(preallmean)

xgbclsmean,xgbregmean,lgbclsmean,lgbregmean
3,2.500392,3,2.327374
4,1.949847,3,2.315179
4,3.251905,1,3.333878
3,1.482522,0,1.434231
0,1.476272,1,1.558648
1,1.332238,2,1.853177


In [53]:
end_time = Sys.time()
time_matrix[4,]=end_time-start_time

In [54]:
result_list=list(preallmin,preallmean,time_matrix)

In [55]:
save(result_list, file = "Monthly_thetaf_opt_pre_result.RData")

In [56]:
time_matrix

user_time,system_time,elapsed_time
13.666330,13.666330,13.666330
4.267360,4.267360,4.267360
1.593232,1.593232,1.593232
13.112766,13.112766,13.112766


## importance

In [57]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_min_cl)

# 打印特征重要性
print(importance_matrix)


              Feature       Gain       Cover  Frequency
               <char>      <num>       <num>      <num>
 1:            length 0.26659128 0.111071073 0.06743982
 2:         linearity 0.12555760 0.097278727 0.08054770
 3:         curvature 0.10650466 0.095388472 0.08902641
 4:         seas_acf1 0.06632128 0.084033551 0.06549069
 5: seasonal_strength 0.04681443 0.061971570 0.06870675
 6:           x_acf10 0.03769340 0.045786107 0.04556086
 7:            x_acf1 0.03617671 0.048179071 0.04926420
 8:             spike 0.03573349 0.057993255 0.05325992
 9:            e_acf1 0.03523531 0.049530992 0.05706072
10:           entropy 0.03457025 0.048549492 0.05335737
11:       diff2_acf10 0.03117070 0.044672587 0.05243154
12:           e_acf10 0.03017505 0.054633780 0.05355229
13:       diff1_acf10 0.03008596 0.043305503 0.05438066
14:             trend 0.02981043 0.049536627 0.05082351
15:        diff2_acf1 0.02906982 0.034697281 0.05379593
16:        diff1_acf1 0.02886974 0.055567935 0.0

In [58]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_min_reg)

# 打印特征重要性
print(importance_matrix)

              Feature       Gain       Cover  Frequency
               <char>      <num>       <num>      <num>
 1:            length 0.31100790 0.076751779 0.05295230
 2:         linearity 0.16051626 0.092025692 0.09981088
 3:         curvature 0.13645492 0.082146811 0.10758563
 4:         seas_acf1 0.05096878 0.078293703 0.05736499
 5: seasonal_strength 0.03621859 0.067002834 0.06093717
 6:           entropy 0.03302910 0.065013483 0.05421307
 7:           x_acf10 0.03253950 0.064215808 0.04202564
 8:             spike 0.03024024 0.067048958 0.06324858
 9:            x_acf1 0.02684082 0.045399470 0.04244589
10:            e_acf1 0.02658752 0.045455904 0.05631435
11:       diff2_acf10 0.02624034 0.056104982 0.04433705
12:             trend 0.02262153 0.039803071 0.09855012
13:       diff1_acf10 0.02246425 0.058452721 0.04391679
14:        diff1_acf1 0.02184422 0.058769275 0.04328640
15:           e_acf10 0.02126241 0.049427455 0.04601807
16:        diff2_acf1 0.02030846 0.038681689 0.0

In [59]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_mean_cl)

# 打印特征重要性
print(importance_matrix)

              Feature       Gain      Cover  Frequency
               <char>      <num>      <num>      <num>
 1:            length 0.18555464 0.09620415 0.06917035
 2:         linearity 0.15221841 0.11036419 0.08635248
 3:         curvature 0.13382296 0.09776398 0.08797251
 4:         seas_acf1 0.06424148 0.07947111 0.06494845
 5: seasonal_strength 0.04809660 0.06843648 0.06715758
 6:           entropy 0.04202919 0.05731885 0.05920471
 7:           x_acf10 0.04132098 0.05356608 0.04403535
 8:             spike 0.03833265 0.04905887 0.05341188
 9:             trend 0.03801838 0.05906972 0.05405007
10:            x_acf1 0.03532467 0.04610664 0.04756996
11:       diff1_acf10 0.03449613 0.04114858 0.05429553
12:            e_acf1 0.03239175 0.04815828 0.05056456
13:           e_acf10 0.03200590 0.04207122 0.05385371
14:        diff2_acf1 0.03152001 0.04507936 0.05297005
15:       diff2_acf10 0.03091066 0.03979112 0.05223368
16:        diff1_acf1 0.02898144 0.04811185 0.04756996
17:       

In [60]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_mean_reg)

# 打印特征重要性
print(importance_matrix)

              Feature       Gain       Cover  Frequency
               <char>      <num>       <num>      <num>
 1:         linearity 0.26574238 0.097446259 0.09775841
 2:         curvature 0.15680772 0.074526029 0.09672063
 3:            length 0.15156167 0.081332618 0.05624741
 4:         seas_acf1 0.05292711 0.087082962 0.05853051
 5:           x_acf10 0.04319565 0.056867338 0.04337900
 6:           entropy 0.03758549 0.057965610 0.05458697
 7: seasonal_strength 0.03633244 0.066155350 0.05998340
 8:             spike 0.03170043 0.052324778 0.05603985
 9:             trend 0.03069693 0.060055111 0.09422997
10:       diff1_acf10 0.03013758 0.057547247 0.05458697
11:            x_acf1 0.02857365 0.059829626 0.04836032
12:            e_acf1 0.02645172 0.062353867 0.05811540
13:       diff2_acf10 0.02263169 0.033000474 0.04088834
14:           e_acf10 0.02150041 0.041949543 0.04566210
15:        diff2_acf1 0.02118806 0.037139222 0.04213367
16:        diff1_acf1 0.01857285 0.053372389 0.0

In [61]:
# 获取特征重要性
importance_matrix <- lgb.importance(lgb_min_cl)

# 打印特征重要性
print(importance_matrix)

              Feature       Gain      Cover  Frequency
               <char>      <num>      <num>      <num>
 1:            length 0.32532723 0.19379535 0.10446667
 2:         linearity 0.13187040 0.11218716 0.08413333
 3:         curvature 0.11397579 0.12128769 0.09706667
 4:         seas_acf1 0.06732843 0.08371738 0.07226667
 5: seasonal_strength 0.03851774 0.05059036 0.06700000
 6:           x_acf10 0.03324583 0.05044074 0.04440000
 7:             spike 0.03126536 0.05062201 0.05320000
 8:            x_acf1 0.02955595 0.03994010 0.04300000
 9:           entropy 0.02938823 0.03222172 0.04993333
10:            e_acf1 0.02834395 0.03767677 0.05046667
11:           e_acf10 0.02546184 0.03776069 0.05180000
12:       diff2_acf10 0.02535980 0.04499722 0.05040000
13:        diff2_acf1 0.02488558 0.02288521 0.04946667
14:             trend 0.02484885 0.03918809 0.04233333
15:       diff1_acf10 0.02267623 0.02313007 0.04633333
16:        diff1_acf1 0.02191523 0.03155722 0.04066667
17:       

In [62]:
# 获取特征重要性
importance_matrix <- lgb.importance(lgb_mean_cl)

# 打印特征重要性
print(importance_matrix)

              Feature       Gain      Cover  Frequency
               <char>      <num>      <num>      <num>
 1:            length 0.22993023 0.19533882 0.10306667
 2:         linearity 0.15887987 0.11893345 0.08333333
 3:         curvature 0.14683391 0.12260474 0.09673333
 4:         seas_acf1 0.06236949 0.08443132 0.06646667
 5: seasonal_strength 0.04554824 0.05624547 0.07066667
 6:           x_acf10 0.03962233 0.03967980 0.04520000
 7:           entropy 0.03748100 0.03581855 0.05320000
 8:            x_acf1 0.03377703 0.04363012 0.05093333
 9:             trend 0.03288874 0.05234929 0.04713333
10:             spike 0.03213322 0.03839415 0.04826667
11:            e_acf1 0.02856906 0.04502030 0.04860000
12:       diff1_acf10 0.02655382 0.02549030 0.04713333
13:       diff2_acf10 0.02578725 0.02761966 0.04800000
14:        diff2_acf1 0.02440031 0.02143501 0.04560000
15:           e_acf10 0.02408209 0.02746997 0.04713333
16:        diff1_acf1 0.02044218 0.03556246 0.03946667
17:       

In [63]:
# 获取特征重要性
importance_matrix <- lgb.importance(lgb_mean_reg)

# 打印特征重要性
print(importance_matrix)

              Feature        Gain       Cover  Frequency
               <char>       <num>       <num>      <num>
 1:         linearity 0.328513477 0.159802729 0.10033333
 2:         curvature 0.182532911 0.129626846 0.11166667
 3:            length 0.180028147 0.191587990 0.12666667
 4:         seas_acf1 0.047098978 0.073677978 0.07000000
 5:           x_acf10 0.039429379 0.061968317 0.05000000
 6:           entropy 0.031167909 0.053419021 0.06333333
 7:             spike 0.025993973 0.058721068 0.05200000
 8: seasonal_strength 0.025699169 0.053353589 0.05200000
 9:             trend 0.021745909 0.043787487 0.04466667
10:            x_acf1 0.018859675 0.036648593 0.04666667
11:            e_acf1 0.017065736 0.022446226 0.04466667
12:       diff2_acf10 0.015735817 0.017217790 0.04200000
13:       diff1_acf10 0.013316361 0.019967196 0.03833333
14:           e_acf10 0.012280200 0.014251843 0.03600000
15:            trough 0.011637132 0.006776539 0.02800000
16:        diff2_acf1 0.0107864

In [64]:
importance_matrix1 <- xgb.importance(model = xgb_mean_cl)
importance_matrix2 <- xgb.importance(model = xgb_mean_reg)
importance_matrix3 <- lgb.importance(lgb_mean_cl)
importance_matrix4 <- lgb.importance(lgb_mean_reg)

importance_matrix1=importance_matrix1[order(importance_matrix1$Feature),]
importance_matrix2=importance_matrix2[order(importance_matrix2$Feature),]
importance_matrix3=importance_matrix3[order(importance_matrix3$Feature),]
importance_matrix4=importance_matrix4[order(importance_matrix4$Feature),]

importance_matrix_mean=(importance_matrix1[,2:4]+importance_matrix2[,2:4]+importance_matrix3[,2:4]+importance_matrix4[,2:4])/4

importance=cbind(importance_matrix1[,1],importance_matrix_mean)

write.csv(importance,'M_thetaf_imp_mean.csv')

In [65]:
importance

Feature,Gain,Cover,Frequency
<chr>,<dbl>,<dbl>,<dbl>
curvature,0.15499938,0.106130399,0.09827328
diff1_acf1,0.01941064,0.041002749,0.04005357
diff1_acf10,0.02612597,0.036038329,0.04858729
diff2_acf1,0.02197370,0.030629876,0.04434260
diff2_acf10,0.02376635,0.029407261,0.04578050
e_acf1,0.02611957,0.044494668,0.05048666
e_acf10,0.02246715,0.031435645,0.04566229
entropy,0.03706590,0.051130508,0.05758125
length,0.18676867,0.141115895,0.08878777


In [67]:
# 使用saveRDS保存模型到.rds文件
saveRDS(xgb_mean_reg, file = "M_thetafxgbreg.rds")
saveRDS(xgb_mean_cl, file = "M_thetafxgbcls.rds")
saveRDS(lgb_mean_reg, file = "M_thetaflgbreg.rds")
saveRDS(lgb_mean_cl, file = "M_thetaflgbcls.rds")